In [ ]:
import requests

# Go to https://www.yelp.com/developers/
# create an app, and get the key
key = 'iI4dsdXt-g13ESGVicCEP7g8svwE657yNKQtQx0UXtrjgteJuPjddz_RYf92YIWiUJxpMx_To53E2hXoMQmnpOSc3Jws0L7SAQTH2Qdno9XFlOih236mUlx6AIEhW3Yx'
auth_header = {'Authorization': 'Bearer ' + key}

# We demo the "Business Search" part of the Yelp API
# See https://www.yelp.com/developers/documentation/v3/business_search 
# for documentation
url = 'https://api.yelp.com/v3/businesses/search'

# We search for "Village Taverna in New York"
# Check the documentation for other parameters
parameters = {
    "term": "village taverna",
    "location": "new york, ny",
    "limit": 50
}

# Issue the authenticated request
resp = requests.get(url, headers=auth_header, params=parameters)
data = resp.json()

In [ ]:
data

In [ ]:
# We get back the answer. It contains three keys
# The main one is the "business", which contains a list of the results
data.keys()

In [ ]:
# We can put the results directly into a dataframe
# The issue is that some columns (e.g location) are composite and
# instead of containing values, the cells contain *dictionaries*
import pandas as pd
df = pd.DataFrame(data['businesses'])
df.head(5)

In [ ]:
# In principle, the json_normalize command should be 
# able to parse the JSON response and create a dataframe
# http://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.io.json.json_normalize.html

# For some reason (to be debugged) it does not work. 


In [ ]:
import pandas as pd
df = pd.DataFrame(data['businesses'])

# We take the dictionaries in the location and coordinates columns
# and create new dataframes for both of them, with each element of the dictionary
# now being a column
address_df = df.location.apply(pd.Series)
coordinates_df = df.coordinates.apply(pd.Series)

# We remove location and coordinates from the original dataframe
df.drop('location', axis='columns', inplace=True)
df.drop('coordinates', axis='columns', inplace=True)

# We also drop columns that we do not need
df.drop('categories', axis='columns', inplace=True)
df.drop('transactions', axis='columns', inplace=True)
df.drop('image_url', axis='columns', inplace=True)

# Put together the original dataframe, the addresses, and the coordinates dataframes
# (We concatenate the 3 dataframes bu putting them all next to each other)
new_df = pd.concat([df, address_df, coordinates_df], axis='columns')
new_df
